In [1]:
"""A very simple MNIST classifier.
See extensive documentation at
https://www.tensorflow.org/get_started/mnist/beginners
"""
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import argparse
import sys

from tensorflow.examples.tutorials.mnist import input_data

import tensorflow as tf

FLAGS = None


E:\Anaconda\envs\python3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
# Import data
data_dir = 'MNIST_data/'
#下载数据  并且读取
mnist = input_data.read_data_sets(data_dir, one_hot=True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [3]:
#通过它，你可以更加灵活地构建你的代码。它能让你在运行图的时候，插入一些计算图，这些计算图是由某些操作(operations)构成的
sess = tf.InteractiveSession()

In [4]:
#定义占位符
x = tf.placeholder("float", shape=[None, 784])
#float  类型
#shape=[None, 784]  n行  784列

y_ = tf.placeholder("float", shape=[None, 10])
#float  类型
#shape=[None, 10]   n行  10列

In [5]:
#定义变量
W = tf.Variable(tf.zeros([784,10]))
#tf.zeros([784,10])   W的维度是[784，10]，因为我们想要用784维的图片向量乘以它以得到一个10维的证据值向量
b = tf.Variable(tf.zeros([10]))
#b  偏置量  和要得到的结果一样

In [6]:
#初始化所有变量和常量
sess.run(tf.initialize_all_variables())

Instructions for updating:
Use `tf.global_variables_initializer` instead.


In [7]:
#把向量化后的图片x和权重矩阵W相乘，加上偏置b，然后计算每个分类的softmax概率值
y = tf.nn.softmax(tf.matmul(x,W) + b)
#tf.matmul(x,W) + b  感知器

In [8]:
#损失函数是目标类别和预测类别之间的交叉熵
cross_entropy = -tf.reduce_sum(y_*tf.log(y))

In [9]:
#算出梯度值
train_step = tf.train.GradientDescentOptimizer(0.01).minimize(cross_entropy)
#0.01  学习率
#cross_entropy  交叉熵

In [10]:
#迭代1000次
for i in range(1000):
  #抓取训练集中的50个处理节点
  batch = mnist.train.next_batch(50)
  #训练数据
  train_step.run(feed_dict={x: batch[0], y_: batch[1]})
  #feed_dict={x: batch[0], y_: batch[1]}  一个列表
  #x: batch[0]  相对应
  # y_: batch[1]

In [11]:
#预测是否真实标签匹配(索引位置一样表示匹配)
correct_prediction = tf.equal(tf.argmax(y,1), tf.argmax(y_,1))

In [12]:
#计算出平均值
accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))
#tf.cast(correct_prediction       [True, False, True, True] 会变成 [1,0,1,1]

In [13]:
#输出测试集的分数  需要优化
print (sess.run(accuracy, feed_dict={x: mnist.test.images, y_: mnist.test.labels}))

0.9064


In [14]:
def weight_variable(shape):
  #产生正太分布
  initial = tf.truncated_normal(shape, stddev=0.1)
  #shape表示生成张量的维度
  #stddev=0.1  标准差
  
  #返回一个可变量的值
  return tf.Variable(initial)

def bias_variable(shape):
  #定义常量
  initial = tf.constant(0.1, shape=shape)
  # 0.1  值
  #shape=shape  指定形状

  #返回一个可变量的值
  return tf.Variable(initial)

方法解释                                                                                                                                       
卷积:                                                                                                                                          
tf.nn.conv2d(input, filter, strides, padding, use_cudnn_on_gpu=None, name=None)   

第一个参数input：指需要做卷积的输入图像，它要求是一个Tensor，具有[batch, in_height, in_width, in_channels]这样的shape，具体含义是[训练时一个batch的图片数量, 图片高度, 图片宽度, 图像通道数]，注意这是一个4维的Tensor，要求类型为float32和float64其中之一

第二个参数filter：相当于CNN中的卷积核，它要求是一个Tensor，具有[filter_height, filter_width, in_channels, out_channels]这样的shape，具体含义是[卷积核的高度，卷积核的宽度，图像通道数，卷积核个数]，要求类型与参数input相同，有一个地方需要注意，第三维in_channels，就是参数input的第四维

第三个参数strides：卷积时在图像每一维的步长，这是一个一维的向量，长度4

第四个参数padding：string类型的量，只能是"SAME","VALID"其中之一，这个值决定了不同的卷积方式 当其为‘SAME’时，表示卷积核可以停留在图像边缘  

第五个参数：use_cudnn_on_gpu:bool类型，是否使用cudnn加速，默认为true

池化:                                                                                                                                          
tf.nn.max_pool(value, ksize, strides, padding, name=None) 

value：池化的输入，一般池化层接在卷积层的后面，所以输出通常为feature map。feature map依旧是[batch, in_height, in_width, in_channels]这样的参数。

ksize：池化窗口的大小，参数为四维向量，通常取[1, height, width, 1]，因为我们不想在batch和channels上做池化，所以这两个维度设为了1。ps：估计面tf.nn.conv2d中stries的四个取值也有              相同的意思。                                                                                   

stries：步长，同样是一个四维向量。

padding：填充方式同样只有两种不重复了。


In [15]:
def conv2d(x, W):
  return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')

def max_pool_2x2(x):
  return tf.nn.max_pool(x, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')

In [16]:
#把权重变为静态分布
W_conv1 = weight_variable([5, 5, 1, 32])
#拿到偏置量,并且指定形状
b_conv1 = bias_variable([32])

In [17]:
#调整x的形状  (x是读取的单张图片  x = tf.placeholder("float", shape=[None, 784]))
x_image = tf.reshape(x, [-1,28,28,1])

In [18]:
#调用relu算法
h_conv1 = tf.nn.relu(conv2d(x_image, W_conv1) + b_conv1)
#conv2d(x_image, W_conv1)  构造卷积卷积
#把算法池化
h_pool1 = max_pool_2x2(h_conv1)

In [19]:
#把权重变为静态分布
W_conv2 = weight_variable([5, 5, 32, 64])
#拿到偏置量,并且指定形状
b_conv2 = bias_variable([64])
#调用relu算法
h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2) + b_conv2)
#conv2d(h_pool1, W_conv2)  构造卷积卷积

#把算法池化
h_pool2 = max_pool_2x2(h_conv2)

In [20]:
#把权重变为静态分布
W_fc1 = weight_variable([7 * 7 * 64, 1024])
#拿到偏置量,并且指定形状
b_fc1 = bias_variable([1024])
#调整池化的形状
h_pool2_flat = tf.reshape(h_pool2, [-1, 7*7*64])
#调用relu算法
h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)

tf.nn.dropout(x, keep_prob, noise_shape=None, seed=None, name=None)

根据给出的keep_prob参数，将输入tensor x按比例输出。

x                 :  输入tensor


keep_prob    :  float类型，每个元素被保留下来的概率


noise_shape  : 一个1维的int32张量，代表了随机产生“保留/丢弃”标志的shape。


seed             : 整形变量，随机数种子。


name            : 名字，没啥用。

In [21]:
#定义一个占位符
keep_prob = tf.placeholder("float")
h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)

In [22]:
#把权重变为静态分布
W_fc2 = weight_variable([1024, 10])
#拿到偏置量,并且指定形状
b_fc2 = bias_variable([10])

#进行softmax
y_conv=tf.nn.softmax(tf.matmul(h_fc1_drop, W_fc2) + b_fc2)
#tf.matmul(h_fc1_drop, W_fc2) + b_fc2  感知器

In [23]:
#拿到交叉熵
cross_entropy = -tf.reduce_sum(y_*tf.log(y_conv))
#AdamOptimizer通过使用动量（参数的移动平均数）来改善传统梯度下降，促进超参数动态调整。我们可以通过创建标签错误率的摘要标量来跟踪丢失和错误率
train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)

correct_prediction = tf.equal(tf.argmax(y_conv,1), tf.argmax(y_,1))
#tf.argmax 是一个非常有用的函数，它能给出某个tensor对象在某一维上的其数据最大值所在的索引值
#tf.equal 来检测我们的预测是否真实标签匹配(索引位置一样表示匹配)

#算出平均值
accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))
#tf.cast   [True, False, True, True] 会变成 [1,0,1,1] ，取平均值后得到 0.75

#初始化所有变量
sess.run(tf.initialize_all_variables())

In [24]:
for i in range(20000):
  #每次迭代拿到50个批处理数据点
  batch = mnist.train.next_batch(50)
  if i%100 == 0:
    #计算出训练集的分数
    train_accuracy = accuracy.eval(feed_dict={ x:batch[0], y_: batch[1], keep_prob: 1.0})
    #keep_prob: 1.0  保存结果  1为保存全部
    print ("step %d, training accuracy %g"%(i, train_accuracy))
  #训练
  train_step.run(feed_dict={x: batch[0], y_: batch[1], keep_prob: 0.5})

step 0, training accuracy 0.04
step 100, training accuracy 0.86
step 200, training accuracy 0.98
step 300, training accuracy 0.96
step 400, training accuracy 0.96
step 500, training accuracy 0.92
step 600, training accuracy 0.98
step 700, training accuracy 0.92
step 800, training accuracy 0.9
step 900, training accuracy 0.96
step 1000, training accuracy 0.98
step 1100, training accuracy 0.94
step 1200, training accuracy 0.96
step 1300, training accuracy 0.96
step 1400, training accuracy 0.98
step 1500, training accuracy 1
step 1600, training accuracy 0.98
step 1700, training accuracy 0.96
step 1800, training accuracy 0.96
step 1900, training accuracy 0.98
step 2000, training accuracy 0.96
step 2100, training accuracy 0.96
step 2200, training accuracy 0.98
step 2300, training accuracy 0.92
step 2400, training accuracy 0.94
step 2500, training accuracy 0.92
step 2600, training accuracy 0.98
step 2700, training accuracy 1
step 2800, training accuracy 0.98
step 2900, training accuracy 1
st

In [25]:
#计算出测试集的分数
print ("test accuracy %g"%accuracy.eval(feed_dict={x: mnist.test.images, y_: mnist.test.labels, keep_prob: 1.0}))

test accuracy 0.9931
